In [1]:
from process_mstar_AConvNet import *

In [3]:
project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))))

mstar_dataset_root = os.path.join(project_root, "datasets/MSTAR")

# Input directory (raw CD structure)
RAW_DATA_DIR = os.path.join(mstar_dataset_root, "mstar_raw_data")

print(RAW_DATA_DIR)

/home/mqtho/datasets/MSTAR/mstar_raw_data


In [10]:
file_path = '/home/mqtho/ATR-Code/datasets/MSTAR/mstar_raw_data/MSTAR-PublicMixedTargets-CD1/MSTAR_PUBLIC_MIXED_TARGETS_CD1/15_DEG/COL1/SCENE1/BTR_60/HB05629.003'

metadata, mag_image = parse_mstar_file(file_path)

In [13]:
metadata

{'PhoenixHeaderLength': '01604',
 'PhoenixSigSize': '00132676',
 'PhoenixSigNum': '0001',
 'PhoenixHeaderCallingSequence': '',
 'HeaderVersionNumber': '8CM',
 'native_header_length': '0',
 'Filename': 'hb05629.0003',
 'Chip_MD5_CheckSum': 'c26f6c9d4f0ef679d950cff87aa8b535',
 'ParentScene': 'hb05629',
 'NumberOfColumns': '128',
 'NumberOfRows': '128',
 'TargetType': 'btr60_transport',
 'TargetSerNum': 'k10yt7532',
 'TargetAz': '292.483337',
 'TargetRoll': '359.943665',
 'TargetPitch': '359.608215',
 'TargetYaw': '6.784937',
 'DesiredDepression': '15',
 'DesiredGroundPlaneSquint': '-90',
 'DesiredSlantPlaneSquint': '-90',
 'DesiredRange': '4500',
 'DesiredAimpointElevation': '166',
 'MeasuredDepression': '15.027344',
 'MeasuredGroundPlaneSquint': '-85.971390',
 'MeasuredSlantPlaneSquint': '-86.109375',
 'MeasuredRange': '4498',
 'MeasuredAimpointElevation': '165.759003',
 'MeasuredAircraftHeading': '61.000000',
 'MeasuredAircraftAltitude': '1332.000000',
 'RadarMode': 'mode 5 - spot ligh

In [11]:
process_image(mag_image, target_size=(128, 128))

array([[155, 132, 178, ..., 132, 103, 132],
       [158, 155, 167, ..., 157, 107, 118],
       [ 89, 185, 182, ..., 168, 170,  83],
       ...,
       [107,  60,  50, ...,  50, 121,  60],
       [ 60, 115, 115, ..., 107,  83,  38],
       [ 94, 118, 137, ...,  69,  50, 107]], shape=(128, 128), dtype=uint8)

In [15]:
get_all_partitions(file_path)

TypeError: get_all_partitions() missing 2 required positional arguments: 'path_metadata' and 'file_path'

In [19]:
import glob
len(glob.glob('/home/mqtho/ATR-Code/datasets/MSTAR/MSTAR_IMG_JSON/SOC/train/2S1/*.json'))

299

In [5]:
import os
import glob
from collections import defaultdict

base_path = '/home/mqtho/ATR-Code/datasets/MSTAR/MSTAR_IMG_JSON'

# ============ SIMPLE : Récursif ============
print("=== COMPTAGE SIMPLE (RÉCURSIF) ===\n")

png_files = glob.glob(os.path.join(base_path, '**', '*.png'), recursive=True)
json_files = glob.glob(os.path.join(base_path, '**', '*.json'), recursive=True)

print(f"Total PNG  : {len(png_files)}")
print(f"Total JSON : {len(json_files)}")
print(f"Total fichiers : {len(png_files) + len(json_files)}")


# ============ DÉTAILLÉ : Par dossier ============
print("\n=== COMPTAGE PAR DOSSIER ===\n")

stats = defaultdict(lambda: {'png': 0, 'json': 0})

for root, dirs, files in os.walk(base_path):
    png_count = len([f for f in files if f.endswith('.png')])
    json_count = len([f for f in files if f.endswith('.json')])
    
    if png_count > 0 or json_count > 0:
        relative_path = os.path.relpath(root, base_path)
        stats[relative_path] = {'png': png_count, 'json': json_count}
        print(f"{relative_path:<50} PNG: {png_count:<5} JSON: {json_count:<5}")


# ============ TABLEAU BIEN FORMATÉ ============
print("\n=== TABLEAU FORMATÉ ===\n")
print(f"{'Chemin':<50} {'PNG':<8} {'JSON':<8} {'Total':<8}")
print("-" * 75)

total_png = 0
total_json = 0

for path in sorted(stats.keys()):
    png_count = stats[path]['png']
    json_count = stats[path]['json']
    total = png_count + json_count
    print(f"{path:<50} {png_count:<8} {json_count:<8} {total:<8}")
    total_png += png_count
    total_json += json_count

print("-" * 75)
print(f"{'TOTAL':<50} {total_png:<8} {total_json:<8} {total_png + total_json:<8}")


# ============ PAR DATASET ET SPLIT ============
print("\n=== PAR DATASET ET SPLIT ===\n")

dataset_stats = defaultdict(lambda: defaultdict(lambda: {'png': 0, 'json': 0}))

for root, dirs, files in os.walk(base_path):
    relative_path = os.path.relpath(root, base_path)
    parts = relative_path.split(os.sep)
    
    if len(parts) >= 2:
        dataset = parts[0]
        split = parts[1]
        
        png_count = len([f for f in files if f.endswith('.png')])
        json_count = len([f for f in files if f.endswith('.json')])
        
        dataset_stats[dataset][split]['png'] += png_count
        dataset_stats[dataset][split]['json'] += json_count

for dataset in sorted(dataset_stats.keys()):
    print(f"\n{dataset}:")
    print(f"  {'Split':<10} {'PNG':<10} {'JSON':<10} {'Total':<10}")
    print(f"  " + "-" * 45)
    
    for split in sorted(dataset_stats[dataset].keys()):
        png = dataset_stats[dataset][split]['png']
        json = dataset_stats[dataset][split]['json']
        total = png + json
        print(f"  {split:<10} {png:<10} {json:<10} {total:<10}")
    
    total_dataset_png = sum(s['png'] for s in dataset_stats[dataset].values())
    total_dataset_json = sum(s['json'] for s in dataset_stats[dataset].values())
    print(f"  {'TOTAL':<10} {total_dataset_png:<10} {total_dataset_json:<10} {total_dataset_png + total_dataset_json:<10}")


# ============ VÉRIFICATION (PNG = JSON) ============
print("\n=== VÉRIFICATION ===\n")
if total_png == total_json:
    print(f"✅ OK : {total_png} PNG et {total_json} JSON")
else:
    print(f"⚠️  ATTENTION : {total_png} PNG mais {total_json} JSON")
    print(f"   Différence : {abs(total_png - total_json)}")

=== COMPTAGE SIMPLE (RÉCURSIF) ===

Total PNG  : 18220
Total JSON : 18220
Total fichiers : 36440

=== COMPTAGE PAR DOSSIER ===

OUTLIER/train/confuser/2S1                         PNG: 299   JSON: 299  
OUTLIER/train/confuser/ZIL131                      PNG: 299   JSON: 299  
OUTLIER/train/known/BTR70                          PNG: 233   JSON: 233  
OUTLIER/train/known/BMP2                           PNG: 233   JSON: 233  
OUTLIER/train/known/T72                            PNG: 232   JSON: 232  
OUTLIER/test/confuser/2S1                          PNG: 274   JSON: 274  
OUTLIER/test/confuser/ZIL131                       PNG: 274   JSON: 274  
OUTLIER/test/known/BTR70                           PNG: 196   JSON: 196  
OUTLIER/test/known/BMP2                            PNG: 195   JSON: 195  
OUTLIER/test/known/T72                             PNG: 196   JSON: 196  
EOC-1/train/BRDM2                                  PNG: 298   JSON: 298  
EOC-1/train/ZSU234                                 PNG: 29